In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentence-transformers
! pip install transformers['sentencepiece']

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import os
import gzip
import json
import torch
import pandas as pd
from transformers import pipeline

In [ ]:
from transformers import pipeline


ner_pipeline = pipeline("ner", model="NlpHUST/ner-vietnamese-electra-base", tokenizer="NlpHUST/ner-vietnamese-electra-base", 
               aggregation_strategy="simple")

Downloading:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

Downloading:   0%|          | 0.00/794 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

## **Reader**

In [ ]:
reader = pipeline('question-answering', model='ancs21/xlm-roberta-large-vi-qa', tokenizer='ancs21/xlm-roberta-large-vi-qa')
# reader = pipeline('question-answering', model='nguyenvulebinh/vi-mrc-base', tokenizer='nguyenvulebinh/vi-mrc-base')

Downloading:   0%|          | 0.00/789 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/179 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

## **Anvil**

In [ ]:
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 6.4 MB/s 
     |████████████████████████████████| 51 kB 226 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=800393d0357c7fbe9cd83a1106e8d37547f597a318c49cce77462e5e322e6f0d
  Stored in directory: /root/.cache/pip/wheels/ea/f9/a1/34e2943cce3cf7daca304bfc35e91280694ced9194a487ce2f
Successfully built ws4py


In [ ]:
import anvil.server
anvil.server.connect('JIHIFFI4OTSU5G7R75U5KA6P-VXVRAFVWEL3OX27P')

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [ ]:
!pip install timeout-decorator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for timeout-decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5028 sha256=7dadf42186882af450b9da1b6cf72d4cfedae7bc97048047ccc5b30feed7eb1d
  Stored in directory: /root/.cache/pip/wheels/38/05/4e/161d1463ca145ec1023bd4e5e1f31cbf9239aa8f39a2a2b643
Successfully built timeout-decorator


In [ ]:
import time
def retriever(query, hits, num_ret=5, cross=True):
  start_time = time.time()
  ##### Sematic Search #####
  # question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
  # question_embedding = question_embedding.cuda()
  # hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
  # hits = hits[0]

  #### Elastic search ####

  ##### Re-Ranking #####
  # cross_inp = [[query, passages['passage'][hit['corpus_id']]] for hit in hits]
  if cross:
    cross_inp = [[query, hit['_source']['text']] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)

  # return [passages['passage'][hit['corpus_id']].replace("\n", " ") for hit in hits[:num_ret]]
  return [hit['_source']['text'] for hit in hits[:num_ret]], (time.time() - start_time)

In [ ]:
import time
import timeout_decorator
import string

@anvil.server.callable
def ner(query=''):
    res = ner_pipeline(query)
    return [i['word'] for i in res]

@anvil.server.callable
def retrieve(query='', hits=[], num_ret=10, cross=True):
  return retriever(query, hits, num_ret=10, cross=cross)


@anvil.server.callable
def read(query='', contexts=[''], timeout = 20):
  start_time = time.time()

  @timeout_decorator.timeout(timeout, use_signals=False)
  def qa(question = '', context = ''):
    """Read through each context searching for candidate answer
    """
    return reader(question=question, context=context,                   
                    stride=64,
                    max_seq_len=256,
                    max_length=256,
                    padding="longest",
                    truncation="only_second",
                    return_overflowing_tokens=True,
                    return_offset_mapping=True
                  )


  def preprocess(x, remove_puncts=True):
    """ Clean answer
    """
    x = x.replace("\n", " ")
    x = " ".join(x.split())
    if remove_puncts:
        x = "".join([i for i in x if i not in string.punctuation])
    return x
    
  def get_candidate_answers(query='', contexts ='', thrsh = 0.1):
    """ Read through all contexts
    """
    answers = []
    for context, question in zip(contexts, [query]*len(contexts)):
      answer = {'score': 0.0, 'start': 0, 'end': 0, 'answer': ''}
      try: answer = qa(question, context)
      except:  pass
      answer.update({"context": context})
      answers.append(answer)

    answers = sorted(answers, key=lambda x: x['score'], reverse=True)
    answers = [{'score': ans['score'], 'start': ans['start'], 'end': ans['end'], 'answer': preprocess(ans['answer']), 'context':ans['context']}
              for ans in answers if ans['score'] > thrsh]
    return answers

  def voter(candidates = []):
    """ Marjority vote: hard vote for the best answer. If tie, soft vote from the hard-voted candidates.
    """

    if len(candidates)==0:
      return {'score': 0.0, 'start': 0, 'end': 0, 'answer': '', 'context':''}, candidates, (time.time() - start_time)

    counter = {}
    for can in candidates:
        counter[can['answer']] = counter.get(can['answer'], 0)+1

    most_fq = max(counter.items(), key=lambda x:x[1])[1]
    answers = [candidate for candidate in candidates if counter[candidate['answer']] == most_fq]
    if len(answers)>1:
      score = {}
      for ans in answers:
        score[ans['answer']] = score.get(ans['answer'],0)+ans['score']/most_fq
      
      best_can = max(score.items(), key=lambda x:x[1])[0]
      answers = [ans for ans in answers if ans['answer'] == best_can]

    return sorted(answers, key=lambda x:x['score'], reverse=True)[0]
    

   #### Stuffs happen here ### 
  candidates = get_candidate_answers(query, contexts)
  answers = voter(candidates)
  
  return answers, candidates, (time.time() - start_time)